In [3]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
import os
import math
import pandas
import scipy

In [4]:
def meanColumn(a): # function used in centralizeColumn
    '''finding the mean of X to centralize'''
    (row,column)=np.shape(a)
    mean=[]
    for i in range (0,column):
        temp=0
        for j in range(0,row):
            temp+=a[j][i]
        mean.append(temp/row)
    return mean #1x(column)

def sdColumn(a): # function used in centralizedColumn
    '''finding the sd of X to centralize'''
    (row,column)=np.shape(a)
    sd=[]
    for j in range(0,column):
        sd.append(np.std(np.array(a)[:,j]))
    return sd #1xcolumn

def centralizeColumn(a): # input is X
    '''centralizing the values in a matrix'''
    (row,column)=np.shape(a)
    mean=meanColumn(a)
    sd=sdColumn(a)
    newMean=[]
    newSD=[]
    for i in range(0,row):
        newMean.append(mean)
        newSD.append(sd)
    centralized=np.zeros((row,column))
    for i in range(0,row):
        for j in range(0,column):
            centralized[i][j]=((a[i][j]-newMean[i][j])/newSD[i][j])
    return centralized

def attach(attachX): # input is centralized X
    '''attaching ones to the first column'''
    (N,d)=np.shape(attachX)
    tempX=np.ones((N,1))
    attachedX=np.concatenate((tempX,attachX),axis=1)
    return attachedX

'''define X,Y,and D and refining X'''
N=int(input('Enter the number of samples\n'))
d=int(input('Enter the number of dimensions\n'))
q=1
X=np.random.rand(N,d)
Y=np.random.rand(N,q)
D=np.random.rand(d+1,d+1)
X=centralizeColumn(X)
X=attach(X)
#Lambda=??
#rho=1/Lambda

theLambda=np.arange(1,301,1)
theLambda=np.square(theLambda)



Enter the number of samples
4
Enter the number of dimensions
4


In [6]:
def shrinkageOperator(shrinkV,shrinkLambda,shrinkRho): # function used in ADMM
    '''defining the shrinkage operator'''
    if shrinkV>=shrinkLambda*shrinkRho:
        shrinkValue=shrinkV-(shrinkLambda*shrinkRho)
    elif shrinkV==shrinkLambda*shrinkRho:
        shrinkValue=0
    elif shrinkV<=shrinkLambda*shrinkRho:
        shrinkValue=shrinkV+(shrinkLambda*shrinkRho)
    return shrinkValue



In [7]:
def ADMM(admmX,admmY,admmGamma,admmTau,admmLambda,admmRho,admmD,N=N,d=d): # function used in ADMMiteration
    '''defining beta'''
    toBeInversed=np.matmul(admmX.T,admmX)+(np.multiply((1/(2*admmRho)),np.matmul(admmD.T,admmD))) # (d+1)x(d+1)
    toMultiply_beta=np.linalg.inv(toBeInversed) #(d+1)x(d+1)
    subtracted=np.subtract(admmGamma,admmTau) #(d+1)x1
    toMultiplyAgain_beta=np.matmul(admmX.T,admmY)+np.multiply((1/(2*admmRho)),(np.matmul(admmD.T,subtracted)))#(d+1)x1
    newBeta=np.matmul(toMultiply_beta,toMultiplyAgain_beta) #(d+1)x1
    
    '''defining gamma'''
    DB=np.matmul(admmD,newBeta) #(d+1)x1
    newGamma=np.zeros(((d+1),1))
    for i in range(0,d+1):
        shrinkageInput=DB[i]+admmTau[i]
        newGamma[i]=shrinkageOperator(shrinkageInput,admmLambda,admmRho)
        
    '''defining tau'''
    newTau=DB
    newTau=np.subtract(np.add(admmTau,DB),newGamma)
    
    return newBeta,newGamma,newTau

Beta,Gamma,Tau=ADMM(X,Y,np.random.rand((d+1),1),np.random.rand((d+1),1),3,2,np.random.rand(d+1,d+1))
Beta2,Gamma2,Tau2=ADMM(X,Y,Gamma,Tau,3,2,np.random.rand(d+1,d+1))
Beta3,Gamma3,Tau3=ADMM(X,Y,Gamma2,Tau2,3,2,np.random.rand(d+1,d+1))



In [8]:
def ADMMiteration(iterX,iterY,iterLambda,iterRho,iterD,N=N,d=d,iterNum=10000): # inputs: attachedX, Y, Lambda, Rho, D
    flag=0
    allTau=[]
    allBeta=[]
    allGamma=[]

    initialTau=np.random.rand((d+1),1)
    initialGamma=np.random.rand((d+1),1)
    initialBeta,e,w=ADMM(iterX,iterY,initialGamma,initialTau,iterLambda,iterRho,iterD)
    
    allTau.append(initialTau)
    allGamma.append(initialGamma)
    allBeta.append(initialBeta)
 
    i=1
    while True:
        tempBeta,tempGamma,tempTau=ADMM(iterX,iterY,allGamma[i-1],allTau[i-1],iterLambda,iterRho,iterD)
        allTau.append(tempTau)
        allGamma.append(tempGamma)
        allBeta.append(tempBeta)


        errorTau=np.abs(np.subtract(allTau[i],allTau[i-1]))
        errorBeta=np.abs(np.subtract(allBeta[i],allBeta[i-1]))
        errorGamma=np.abs(np.subtract(allGamma[i],allGamma[i-1]))
        
        i+=1
        
        #if any(errorTau)<10**(10) or any(errorGamma)<10**(10) or any(errorBeta)<10**(10):
        #    flag=0
        #else:
        #    flag=1
            
        if i==iterNum:
            break
        
    return allBeta[1:],allGamma[1:],allTau[1:]

    
iteratedBeta,iteratedGamma,iteratedTau=ADMMiteration(X,Y,3,2,np.random.rand((d+1),(d+1)))   

In [ ]:
def truncateBeta(truncBeta): # input: the last value of beta obtained from ADMMiteration
    for i in range(len(truncBeta)):
        if abs(truncBeta[i])<10**(-5):
            truncBeta[i]=0
    return truncBeta 



In [9]:
def returnCoordinates(returnBeta): # input: the latest iterated beta value after truncation
    coordinates=[]
    for j in range(0,len(returnBeta)):
        if returnBeta[j]!=0:
            coordinates.append(j+1)
    print ('the non-vanishing coordinates are:\n' )
    print (coordinates)
    return coordinates # output: the coordinates for which beta is not zero

meanForY=np.multiply(X.T,iteratedBeta[-1]) 

'''before proceeding next, define theLambda and find the values of beta for all those lambda values''' 

'before proceeding next, define theLambda and find the values of beta for all those lambda values'

In [ ]:
'''plots for the question'''
def nonVanishingCoeff(recoverLambdaB=lambdaB): # input: the big array which has the b values for all lambda values
    toPlot=[]
    for k in range(0,len(recoverLambdaB)):
        w=0
        for j in range(0,len(recoverLambdaB[0])):
            if recoverLambdaB[k][j]!=0:
                w+=1
    toPlot.append(w)   #output: number of non-vanishing elements in b==> to plot against lambda 
    return toPlot
    
def plotB(jValue,LambdaB=lambdaB): # input: the big array which has b values for all lambda values and j=21,36,200
    toBePlotted=[]
    for k in range(0,len(LambdaB)):
        toBePlotted.append(LambdaB[k][j]) # plot against lambda
        
    return toBePlotted

def increment(j,LambdaB=lambdaB,TheLambda=theLambda): # input: the big array of b and j=21,36,200
    toPlot=[]
    for k in range(0,len(LambdaB)-1):
        temp=LambdaB[k+1][j]-Lambda[k][j]
        temp1=theLambda[k+1]-thelambda[k]
        temp3=temp/temp1
        toPlot.append(temp3) # note that this is one element less than that of theLambda